# 11th Day of Python 4 DIP

> Contents:
> 1. Corner detection Harris methodology
> 2. Detection of lines and curves Hough transform
> 3. Mathematical morphology
 

Execute the cells to gain outputs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage as SK
from Tools import *
%matplotlib inline

### Unsharp Masking

In [ ]:
Camera = SK.io.imread("./Figures/cameraman.jpg",as_gray=True)
Res1 = SK.filters.unsharp_mask(Camera,radius=1,amount=1.0)
Res2 = SK.filters.unsharp_mask(Camera,radius=1,amount=2.0)
Res3 = SK.filters.unsharp_mask(Camera,radius=5,amount=2.0)
ShowTimeMul((Camera,Res1),Im_title=["Original","Enhanced 1-1"],With_Hist=False,cmaps=["gray"]*2,colorbar=False)
ShowTimeMul((Res2,Res3),Im_title=["Enhanced 1-2","Enhanced 5-2"],With_Hist=False,cmaps=["gray"]*2,colorbar=False)

### Corner detection Harris

1. Compute $x$ and $y$ derivatives ($I_x, I_y$) of an image
2. Compute products of derivatives ($I_x^2, I_y^2, I_{xy}$) at each pixel
3. Compute matrix $M$ at each pixel, where
$$
M = \sum_{x,y} w(x,y)
    \begin{bmatrix}
        I_{x}^2 & I_{x}I_{y} \\
        I_{x}I_{y} & I_{y}^2
    \end{bmatrix}
$$
4. Compute corner response $R=Det(M)-k(Trace(M)^2)$ at each pixel
5. Output corner response map $R(x,y)$


In [ ]:
## Lets create an awesome image
r,c = SK.draw.ellipse(160,175,10,100)
Transform = SK.transform.AffineTransform(scale=(1.3,1.1),rotation=1,shear=0.7,translation=(110,30))
Image = SK.transform.warp(SK.data.checkerboard()[:90,:90],Transform.inverse,output_shape=(200,310))
Image[r,c] = 1.0
Image[30:80,200:250] = 1.0
Image[80:130,250:300] = 1.0
ShowTime(Image,With_Hist=False,colorbar=False)

In [ ]:
Har_Image = SK.feature.corner_harris(Image,method='k',k=0.04)
Coordinates = SK.feature.corner_peaks(Har_Image,min_distance=5,threshold_rel=0.04)
Coordinates_Sub = SK.feature.corner_subpix(Har_Image,Coordinates,window_size=13)
plt.imshow(Image,cmap='inferno')
plt.plot(Coordinates[:,1],Coordinates[:,0],"xr")
plt.plot(Coordinates_Sub[:,1],Coordinates_Sub[:,0],"+b")
plt.show()

### Hough transform (Detection of lines)

In [ ]:
## Creation of Lines image
Lines = np.zeros((100,100))
Lines[SK.draw.line(5,35,75,85)] = 1.0
Lines[SK.draw.line(8,65,80,5)] = 1.0
Lines[SK.draw.line(70,95,70,5)] = 1.0

## Create the angles
th = np.linspace(-np.pi/2,np.pi/2,360)
hough, theta, dists = SK.transform.hough_line(Lines,theta=th)
ShowTimeMul((Lines,np.log(1+hough)),Im_title=["Original","Hough Transform"],With_Hist=False,colorbar=False,cmaps=["gray"]*2)

In [ ]:
fig,ax = plt.subplots()
ax.imshow(Lines,cmap="gray")
a,b,c = SK.transform.hough_line_peaks(hspace=hough,angles=theta,dists=dists)
for _, angle,dist in zip(a,b,c):
    (x,y) = dist * np.array([np.cos(angle),np.sin(angle)])
    ax.axline((x,y),slope=np.tan(angle+np.pi/2))
plt.show()

### For probabilistic hough line transform please go to Day10 notebook.

### Hough transform for curves and circular shapes
>> `Circle detection`

In [ ]:
Coins = SK.util.img_as_ubyte(SK.data.coins())
Edges = SK.feature.canny(Coins,sigma=3,low_threshold=10,high_threshold=50)
## Detect circles with radius
Rads = np.arange(10,45,2)
Hough_Circle = SK.transform.hough_circle(Edges,Rads)
Accs,Cx,Cy,Rad_H = SK.transform.hough_circle_peaks(Hough_Circle,Rads,total_num_peaks=15)
Coins = SK.color.gray2rgb(Coins)
fig,ax = plt.subplots()
for yi,xi,r in zip(Cy,Cx,Rad_H):
    Ciry,Cirx = SK.draw.circle_perimeter(yi,xi,r,shape=Coins.shape)
    Coins[Ciry,Cirx] = (200,0,0)

ax.imshow(Coins,cmap='gray')
for i in range(len(Accs)):
    ax.plot(Cx[i],Cy[i],"+b")
    ax.text(Cx[i],Cy[i],'%2.3f'%Accs[i],horizontalalignment='center')



### Hough transform for curves and circular shapes
>> `Ellipse detection`

Code from documentation of scikit-image

In [ ]:
# Load picture, convert to grayscale and detect edges
image_rgb = SK.data.coffee()[0:220, 160:420]
image_gray = SK.color.rgb2gray(image_rgb)
edges = SK.feature.canny(image_gray, sigma=2.0,
              low_threshold=0.55, high_threshold=0.8)

# Perform a Hough Transform
# The accuracy corresponds to the bin size of a major axis.
# The value is chosen in order to get a single high accumulator.
# The threshold eliminates low accumulators
result = SK.transform.hough_ellipse(edges, accuracy=20, threshold=250,
                       min_size=100, max_size=120)
result.sort(order='accumulator')

# Estimated parameters for the ellipse
best = list(result[-1])
yc, xc, a, b = (int(round(x)) for x in best[1:5])
orientation = best[5]

# Draw the ellipse on the original image
cy, cx = SK.draw.ellipse_perimeter(yc, xc, a, b, orientation)
image_rgb[cy, cx] = (0, 0, 255)
# Draw the edge (white) and the resulting ellipse (red)
edges = SK.color.gray2rgb(SK.util.img_as_ubyte(edges))
edges[cy, cx] = (250, 0, 0)

fig2, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(8, 4),
                                sharex=True, sharey=True)

ax1.set_title('Original picture')
ax1.imshow(image_rgb)

ax2.set_title('Edge (white) and result (red)')
ax2.imshow(edges)

plt.show()



### Mathematical Morphology

In [ ]:
Phantom = SK.util.img_as_ubyte(SK.data.shepp_logan_phantom())
ShowTime(Phantom,With_Hist=False,Im_title="Shepp logans Phantom",colorbar=False)

### `Erosion`

In [ ]:
Disk = SK.morphology.disk(11)
Phantom_Er = SK.morphology.erosion(image=Phantom,footprint=Disk)
ShowTimeMul((Phantom,Phantom_Er),figsize=(8,6),With_Hist=False,Im_title=["Original","Erosion"],colorbar=False,cmaps=["gray"]*2)

### `Dilation`

In [ ]:
Phantom_Dil = SK.morphology.dilation(image=Phantom,footprint=Disk)
ShowTimeMul((Phantom,Phantom_Dil),figsize=(8,6),With_Hist=False,Im_title=["Original","Dilation"],colorbar=False,cmaps=["gray"]*2)

### `Opening`

In [ ]:
Phantom_Open = SK.morphology.opening(image=Phantom,footprint=Disk)
ShowTimeMul((Phantom,Phantom_Open),figsize=(8,6),With_Hist=False,Im_title=["Original","Opening"],colorbar=False,cmaps=["gray"]*2)

### `Closing`

In [ ]:
Phantom_Copy = Phantom.copy()
Phantom_Copy[10:30,200:210] = 0
Phantom_Clos = SK.morphology.closing(image=Phantom_Copy,footprint=Disk)
ShowTimeMul((Phantom_Copy,Phantom_Clos),figsize=(8,6),With_Hist=False,Im_title=["Deleted","Closing"],colorbar=False,cmaps=["gray"]*2)

### `Skletonize`

In [ ]:
Horse = SK.data.horse()
Horse_Skel = SK.morphology.skeletonize(Horse==0)
ShowTimeMul((Horse,Horse_Skel),figsize=(8,6),With_Hist=False,Im_title=["Original","Skeleton"],colorbar=False,cmaps=["gray"]*2)

### Bonus effects of Morphology

In [ ]:
Text = SK.io.imread("./Figures/Text.tif")
Text_Dil = SK.morphology.dilation(Text,footprint=SK.morphology.disk(5))
ShowTimeMul((Text,Text_Dil),figsize=(12,6),With_Hist=False,Im_title=["Original","Diltion"],colorbar=False,cmaps=["gray"]*2)